In [ ]:
# 📊 Análise Descritiva Northwind
## Comportamento de Clientes e Vendas

Este notebook apresenta uma análise completa dos dados do Northwind com foco em:
- Comportamento de compra dos clientes
- Performance de vendas
- Análise RFM (Recency, Frequency, Monetary)
- Insights de negócio e visualizações interativas

**Autor:** Projeto de Portfólio Data Analytics  
**Data:** 2026-01-21

In [ ]:
# Imports e Configuração
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

from src.database import DatabaseConnection
from src.data_loader import NorthwindDataLoader
from src.analysis.customer_behavior import CustomerBehaviorAnalysis
from src.analysis.sales_analysis import SalesAnalysis

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

%matplotlib inline

print("✓ Bibliotecas importadas com sucesso!")

## 1. Carregamento de Dados

Conectar ao banco PostgreSQL e carregar todas as tabelas do Northwind.

In [ ]:
# Conectar e carregar dados
db = DatabaseConnection()
loader = NorthwindDataLoader(db)
data = loader.load_all()
sales_df = loader.create_sales_view()

print(f'Total de registros: {len(sales_df)}')
print(f'Clientes: {len(data["customers"])}')
print(f'Pedidos: {len(data["orders"])}')
print(f'Produtos: {len(data["products"])}')
print(f'Categorias: {len(data["categories"])}')

sales_df.head()

## 2. Estatísticas Descritivas

Análise exploratória inicial dos dados.

In [ ]:
# Estatísticas das principais métricas
sales_df[['quantity', 'discount', 'total_price', 'freight']].describe()

## 3. Análise RFM (Recency, Frequency, Monetary)

Segmentação de clientes baseada em:
- **Recency**: Há quanto tempo o cliente fez a última compra
- **Frequency**: Quantas vezes o cliente comprou
- **Monetary**: Quanto o cliente gastou

In [ ]:
# Análise RFM
customer_analysis = CustomerBehaviorAnalysis(sales_df, data['customers'])
rfm_df = customer_analysis.calculate_rfm()

print(f"Total de clientes analisados: {len(rfm_df)}")
print(f"\nSegmentação:")
print(rfm_df['segment'].value_counts())

# Visualizar top 10 clientes
rfm_df.sort_values('rfm_score', ascending=False).head(10)

In [ ]:
# Visualizar distribuição de segmentos
fig, ax = plt.subplots(figsize=(10, 6))
rfm_df['segment'].value_counts().plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Distribuição de Segmentos RFM', fontsize=14, fontweight='bold')
ax.set_xlabel('Segmento')
ax.set_ylabel('Número de Clientes')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 4. Análise de Vendas

Métricas principais de vendas e performance de produtos.

In [ ]:
# Métricas gerais
sales_analysis = SalesAnalysis(sales_df)
metrics = sales_analysis.overall_metrics()

print('='*60)
print('MÉTRICAS PRINCIPAIS DE VENDAS')
print('='*60)
print(f"Receita Total: R$ {metrics['total_revenue']:,.2f}")
print(f"Total de Pedidos: {metrics['total_orders']}")
print(f"Ticket Médio: R$ {metrics['avg_order_value']:.2f}")
print(f"Itens Vendidos: {metrics['total_items_sold']}")
print(f"Produtos Únicos: {metrics['unique_products']}")
print(f"Clientes Únicos: {metrics['unique_customers']}")
print(f"Desconto Médio: {metrics['avg_discount']*100:.1f}%")
print('='*60)

## 5. Top Produtos

Análise dos produtos mais vendidos por receita e quantidade.

In [ ]:
# Top produtos
top_revenue, top_quantity = sales_analysis.product_performance(10)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top por receita
top_revenue.sort_values('revenue').plot(
    x='product_name', y='revenue', kind='barh', ax=axes[0], color='steelblue', legend=False
)
axes[0].set_title('Top 10 Produtos por Receita', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Receita (R$)')
axes[0].set_ylabel('')

# Top por quantidade
top_quantity.sort_values('units_sold').plot(
    x='product_name', y='units_sold', kind='barh', ax=axes[1], color='coral', legend=False
)
axes[1].set_title('Top 10 Produtos por Quantidade', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Unidades Vendidas')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

## 6. Performance por Categoria

Análise de receita e performance por categoria de produtos.

In [ ]:
# Performance por categoria
category_perf = sales_analysis.category_performance()

# Gráfico de pizza
fig, ax = plt.subplots(figsize=(10, 8))
colors = sns.color_palette('Set3', len(category_perf))
ax.pie(category_perf['revenue'], labels=category_perf['category'], autopct='%1.1f%%', colors=colors, startangle=90)
ax.set_title('Distribuição de Receita por Categoria', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Mostrar dados
category_perf

## 7. Conclusões e Próximos Passos

### 🔍 Principais Descobertas:

1. **Segmentação RFM**: Identificamos diferentes perfis de clientes que podem ser trabalhados com estratégias específicas
2. **Performance de Produtos**: Alguns produtos concentram maior parte da receita
3. **Categorias**: Distribuição clara de receita entre categorias
4. **Oportunidades**: Clientes "At Risk" e "Lost" podem ser reativados

### 📝 Recomendações:

- Focar em retenção de clientes Champions e Loyal
- Desenvolver campanhas para clientes em risco
- Otimizar mix de produtos baseado em performance
- Analisar impacto de descontos na lucratividade

### 🚀 Próximos Passos:

- Análise preditiva de churn
- Modelagem de propensão de compra
- Segmentação avançada com clustering
- Dashboard interativo para acompanhamento